In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import os
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D

/home/vt337/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/vt337/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
proj_dir = Path("..") / "Data/r77-mini-data-fortnight"
print(proj_dir.joinpath("input"))

data_dir = proj_dir
input_dir = data_dir.joinpath("input")
fixed_input_dir = input_dir.joinpath("fixed")
temporal_input_dir = input_dir.joinpath("temporal")
target_dir = data_dir.joinpath("target")
fixed_target_dir = target_dir.joinpath("fixed")
temporal_target_dir = target_dir.joinpath("temporal")

../Data/r77-mini-data-fortnight/input


In [3]:
files = list(temporal_target_dir.iterdir())


index_dir = Path("..")/"Index"
files_index = list(index_dir.iterdir())
indices = np.load(files_index[0]).squeeze()

#First hour of data
index = indices[0]
tt = np.load(files[index]).squeeze()
tt.shape

(144, 70, 100, 3)

In [4]:
files_ti = list(temporal_input_dir.iterdir())

In [5]:
#all diff data
data = []

for j in range(360):
    index = indices[j]

    tt = np.load(files[index]).squeeze()
    tt = np.transpose(tt, (0, 2, 3, 1))[:, :, :, :64]  #shape: (144, 100, 3, 64)

    ti = np.load(files_ti[index]).squeeze()
    ti = np.transpose(ti, (0, 2, 1))[:, :, :64]  #shape: (144, 3, 64)

    #ensure ti has the same second dimension as tt
    ti_expanded = np.repeat(ti[:, None, :, :], tt.shape[1], axis=1)  #shape: (144, 100, 3, 64)

    #calculate diff and reshape
    diff_data = tt - ti_expanded  #shape: (144, 100, 3, 64)
    reshaped_tt = diff_data.reshape(-1, 3, 64)  #shape: (14400, 3, 64)

    data.append(reshaped_tt)


data = np.concatenate(data).reshape(-1, 3, 64)
data.shape

(5184000, 3, 64)

In [6]:
def normalise(vector):
    min_val = np.min(vector)
    
    max_val = np.max(vector)
    normalised_vector = (vector - min_val) / (max_val - min_val)
    return normalised_vector, min_val, max_val

In [7]:
files_fixed = list(fixed_input_dir.iterdir())
files_ft = list(fixed_target_dir.iterdir())
#std dev calculated for each grid using fxed target vectors, combined with fixed input vector to give three fixed variables for each grid
#fixed variables all (fixed input repeated 100 times for each grid)

cond = []

for j in range(360):
    
    index = indices[j]
    fi = np.load(files_fixed[index]).squeeze()
    ft = np.load(files_ft[index]).squeeze()

    stdev_orog = np.array([np.std(i) for i in ft[:,:,1]]).reshape(-1,1)
    fixed_input = np.hstack((fi, stdev_orog))
    reshaped_fi = np.repeat(fixed_input, 100, axis=0)
    cond.append(reshaped_fi)


cond = np.concatenate(cond).reshape(-1, 3)
cond.shape

(5184000, 3)

In [8]:
lsf = cond[:,0]
orog = cond[:,1]
stdev_orog = cond[:,2]

In [9]:
#all data
tt_all = np.stack([
    np.transpose(np.load(files[indices[i]]).squeeze(), (0, 2, 3, 1))[:,:,:,:64].reshape((14400, 3, 64), order = 'C')
    for i in range(360)
])

all_data = tt_all.reshape(-1,3,64)
all_data.shape

(5184000, 3, 64)

In [10]:
#has inversion
def has_inversion(temperature_profile, troposphere_height):
    gradient = np.gradient(temperature_profile[:troposphere_height])
    return np.any(np.array(gradient) > 0.0)

temp = all_data[:,2,:]


inversion = np.array([has_inversion(i, 40) for i in temp])
inversion[inversion == False].shape[0]

1220648

In [11]:
indices1 = np.where((inversion == False))[0]
data1 = data[indices1]

indices2 = np.where((inversion == True))[0]
data2 = data[indices2]

print(f'no inversion training data {data1.shape}')
print(f'has inversion training data {data2.shape}')


no inversion training data (1220648, 3, 64)
has inversion training data (3963352, 3, 64)


In [12]:
indices1 = np.where((inversion == False) & (lsf <= 0.2))[0]
trainingdata1 = data[indices1]

indices2 = np.where((inversion == False) & (lsf == 1) & (orog < 0.06))[0]
trainingdata2 = data[indices2]

indices3 = np.where((inversion == False) & (lsf == 1) & (orog > 0.06))[0]
trainingdata3 = data[indices3]

indices4 = np.where((inversion == False) & (lsf < 1) & (lsf > 0.2) & (orog < 0.03))[0]
trainingdata4 = data[indices4]

indices5 = np.where((inversion == False) & (lsf < 1) & (lsf > 0.2) & (orog > 0.03))[0]
trainingdata5 = data[indices5]

indices6 = np.where((inversion == True) & (lsf <= 0.2))[0]
trainingdata6 = data[indices6]

indices7 = np.where((inversion == True) & (lsf == 1) & (orog < 0.06))[0]
trainingdata7 = data[indices7]

indices8 = np.where((inversion == True) & (lsf == 1) & (orog > 0.06))[0]
trainingdata8 = data[indices8]

indices9 = np.where((inversion == True) & (lsf < 1) & (lsf > 0.2) & (orog < 0.03))[0]
trainingdata9 = data[indices9]

indices10 = np.where((inversion == True) & (lsf < 1) & (lsf > 0.2) & (orog > 0.03))[0]
trainingdata10 = data[indices10]

In [13]:
print(f'class1 training data {trainingdata1.shape}, {round(trainingdata1.shape[0]/5184000, 3)*100}')
print(f'class2 training data {trainingdata2.shape},{round(trainingdata2.shape[0]/5184000, 3)*100}')
print(f'class3 training data {trainingdata3.shape}, {round(trainingdata3.shape[0]/5184000, 3)*100}')
print(f'class4 training data {trainingdata4.shape}, {round(trainingdata4.shape[0]/5184000, 3)*100}')
print(f'class5 training data {trainingdata5.shape}, {round(trainingdata5.shape[0]/5184000, 3)*100}')
print(f'class6 training data {trainingdata6.shape}, {round(trainingdata6.shape[0]/5184000, 3)*100}')
print(f'class7 training data {trainingdata7.shape}, {round(trainingdata7.shape[0]/5184000, 3)*100}')
print(f'class8 training data {trainingdata8.shape}, {round(trainingdata8.shape[0]/5184000, 3)*100}')
print(f'class9 training data {trainingdata9.shape}, {round(trainingdata9.shape[0]/5184000, 3)*100}')
print(f'class10 training data {trainingdata10.shape}, {round(trainingdata10.shape[0]/5184000, 3)*100}')

class1 training data (645213, 3, 64), 12.4
class2 training data (114201, 3, 64),2.1999999999999997
class3 training data (60029, 3, 64), 1.2
class4 training data (266266, 3, 64), 5.1
class5 training data (134939, 3, 64), 2.6
class6 training data (2018787, 3, 64), 38.9
class7 training data (425799, 3, 64), 8.200000000000001
class8 training data (191971, 3, 64), 3.6999999999999997
class9 training data (885734, 3, 64), 17.1
class10 training data (441061, 3, 64), 8.5


In [16]:
data_list = [trainingdata1, trainingdata2, trainingdata3,trainingdata4,trainingdata5,trainingdata6, trainingdata7, trainingdata8,trainingdata9,trainingdata10 ]
sum([i.shape[0] for i in data_list]) == data.shape[0]

True

In [18]:

for idx, trainingdata in enumerate(data_list, start = 1): 
    
    model = Unet1D(
        dim = 64,
        dim_mults = (1, 2, 4, 8),
        channels = 3
    )

    diffusion = GaussianDiffusion1D(
        model,
        seq_length = 64,
        timesteps = 100,
        objective = 'pred_v'
    )

    data = normalise(trainingdata)[0]
    training_seq =  torch.from_numpy(data)

    trainer = Trainer1D(
        diffusion,
        dataset = training_seq,
        train_batch_size = 10, #set batch size here (take 100 samples, one grid)
        train_lr = 1e-4,
        train_num_steps = 100000,         # total training steps (1000)
        gradient_accumulate_every = 2,    # gradient accumulation steps
        ema_decay = 0.995,                # exponential moving average decay
        amp = True,                       # turn on mixed precision
    )
    trainer.train()

    # after a lot of training

    sampled_seq = diffusion.sample(batch_size = 10000)
    torch.save(sampled_seq, f"sampled_seq{idx}.pt") 

/home/vt337/.local/lib/python3.10/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
loss: 0.0008: 100%|██████████| 100000/100000 [1:34:48<00:00, 17.58it/s]


training complete


loss: 0.0135: 100%|██████████| 100000/100000 [1:35:06<00:00, 17.52it/s]


training complete


loss: 0.0056: 100%|██████████| 100000/100000 [1:35:15<00:00, 17.50it/s]


...

loss: 0.0022: 100%|██████████| 100000/100000 [1:36:40<00:00, 17.24it/s]


training complete


loss: 0.0074: 100%|██████████| 100000/100000 [1:37:50<00:00, 17.03it/s]


training complete


sampling loop time step: 100%|██████████| 100/100 [00:23<00:00,  4.28it/s]


In [19]:
sampled_sequences_list = []

for i in range(1, 11):  
    sampled_seq = torch.load(f"sampled_seq{i}.pt")
    sampled_sequences_list.append(sampled_seq.cpu().numpy())

sampled_sequences = np.array(sampled_sequences_list)

In [20]:
folder = 'Samples_cond_diffs_A'
if not os.path.exists(folder):
    os.makedirs(folder)

file_path = os.path.join(folder, 'sample.npy')
np.save(file_path, sampled_sequences)

In [11]:
indices1 = np.where((inversion == False) & (lsf <= 0.2))[0]
trainingdata1 = data[indices1]

indices2 = np.where((inversion == False) & (lsf == 1) & (stdev_orog < 0.025))[0]
trainingdata2 = data[indices2]

indices3 = np.where((inversion == False) & (lsf == 1) & (stdev_orog > 0.025))[0]
trainingdata3 = data[indices3]

indices4 = np.where((inversion == False) & (lsf < 1) & (lsf > 0.2) & (stdev_orog < 0.03))[0]
trainingdata4 = data[indices4]

indices5 = np.where((inversion == False) & (lsf < 1) & (lsf > 0.2) & (stdev_orog > 0.03))[0]
trainingdata5 = data[indices5]

indices6 = np.where((inversion == True) & (lsf <= 0.2))[0]
trainingdata6 = data[indices6]

indices7 = np.where((inversion == True) & (lsf == 1) & (stdev_orog < 0.025))[0]
trainingdata7 = data[indices7]

indices8 = np.where((inversion == True) & (lsf == 1) & (stdev_orog > 0.025))[0]
trainingdata8 = data[indices8]

indices9 = np.where((inversion == True) & (lsf < 1) & (lsf > 0.2) & (stdev_orog < 0.03))[0]
trainingdata9 = data[indices9]

indices10 = np.where((inversion == True) & (lsf < 1) & (lsf > 0.2) & (stdev_orog > 0.03))[0]
trainingdata10 = data[indices10]

In [12]:
print(f'class1 training data {trainingdata1.shape}, {round(trainingdata1.shape[0]/5184000, 3)*100}')
print(f'class2 training data {trainingdata2.shape},{round(trainingdata2.shape[0]/5184000, 3)*100}')
print(f'class3 training data {trainingdata3.shape}, {round(trainingdata3.shape[0]/5184000, 3)*100}')
print(f'class4 training data {trainingdata4.shape}, {round(trainingdata4.shape[0]/5184000, 3)*100}')
print(f'class5 training data {trainingdata5.shape}, {round(trainingdata5.shape[0]/5184000, 3)*100}')
print(f'class6 training data {trainingdata6.shape}, {round(trainingdata6.shape[0]/5184000, 3)*100}')
print(f'class7 training data {trainingdata7.shape}, {round(trainingdata7.shape[0]/5184000, 3)*100}')
print(f'class8 training data {trainingdata8.shape}, {round(trainingdata8.shape[0]/5184000, 3)*100}')
print(f'class9 training data {trainingdata9.shape}, {round(trainingdata9.shape[0]/5184000, 3)*100}')
print(f'class10 training data {trainingdata10.shape}, {round(trainingdata10.shape[0]/5184000, 3)*100}')

class1 training data (645213, 3, 64), 12.4
class2 training data (96400, 3, 64),1.9
class3 training data (77830, 3, 64), 1.5
class4 training data (278033, 3, 64), 5.4
class5 training data (123172, 3, 64), 2.4
class6 training data (2018787, 3, 64), 38.9
class7 training data (360200, 3, 64), 6.9
class8 training data (257570, 3, 64), 5.0
class9 training data (928967, 3, 64), 17.9
class10 training data (397828, 3, 64), 7.7


In [13]:
data_list = [trainingdata1, trainingdata2, trainingdata3,trainingdata4,trainingdata5,trainingdata6, trainingdata7, trainingdata8,trainingdata9,trainingdata10 ]
sum([i.shape[0] for i in data_list]) == data.shape[0]

True

In [14]:

for idx, trainingdata in enumerate(data_list, start = 1): 
    
    model = Unet1D(
        dim = 64,
        dim_mults = (1, 2, 4, 8),
        channels = 3
    )

    diffusion = GaussianDiffusion1D(
        model,
        seq_length = 64,
        timesteps = 100,
        objective = 'pred_v'
    )

    data = normalise(trainingdata)[0]
    training_seq =  torch.from_numpy(data)

    trainer = Trainer1D(
        diffusion,
        dataset = training_seq,
        train_batch_size = 10, #set batch size here (take 100 samples, one grid)
        train_lr = 1e-4,
        train_num_steps = 10000,         # total training steps (1000)
        gradient_accumulate_every = 2,    # gradient accumulation steps
        ema_decay = 0.995,                # exponential moving average decay
        amp = True,                       # turn on mixed precision
    )
    trainer.train()

    # after a lot of training

    sampled_seq = diffusion.sample(batch_size = 10000)
    torch.save(sampled_seq, f"sampled_seq{idx}.pt")

/home/vt337/.local/lib/python3.10/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
loss: 0.0018: 100%|██████████| 10000/10000 [09:21<00:00, 17.80it/s]


training complete


loss: 0.0083: 100%|██████████| 10000/10000 [09:37<00:00, 17.30it/s]


training complete


loss: 0.0083: 100%|██████████| 10000/10000 [09:55<00:00, 16.79it/s]


...

loss: 0.0024: 100%|██████████| 10000/10000 [09:37<00:00, 17.31it/s]


training complete


loss: 0.0113: 100%|██████████| 10000/10000 [09:51<00:00, 16.91it/s]


training complete


sampling loop time step: 100%|██████████| 100/100 [00:23<00:00,  4.27it/s]


In [15]:
sampled_sequences_list = []

for i in range(1, 11):  
    sampled_seq = torch.load(f"sampled_seq{i}.pt")
    sampled_sequences_list.append(sampled_seq.cpu().numpy())

sampled_sequences = np.array(sampled_sequences_list)

In [16]:
folder = 'Samples_cond_diffs_C'
if not os.path.exists(folder):
    os.makedirs(folder)

file_path = os.path.join(folder, 'sample.npy')
np.save(file_path, sampled_sequences)

: 